In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import random

In [2]:
plt.rcParams["figure.figsize"] = (8,6)

In [3]:
def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc
        
    
def clear_frame(ax=None): 
    # Taken from a post by Tony S Yu
    if ax is None: 
        ax = plt.gca() 
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False) 
    for spine in ax.spines.itervalues(): 
        spine.set_visible(False) 

In [4]:
def foucault_pendulum(x, y, parameters):
    a1, b12, b11, a2, b21, b22 = parameters
    return (a1*x-b12*x*y-b11*x*x, a2*y-b21*x*y-b22*y*y)

In [5]:
def runge_kutta(iterations, initial, step, parameters):
    x_val = [initial[0]]
    y_val = [initial[1]]
    t_val = [0]
    for i in range(round(iterations/step)):
        x1, y1 = competing_species_function(x_val[-1], y_val[-1], parameters)
        x2, y2 = competing_species_function(x_val[-1]+step*x1/2, y_val[-1]+step*y1/2, parameters)
        x3, y3 = competing_species_function(x_val[-1]+step*x2/2, y_val[-1]+step*y2/2, parameters)
        x4, y4 = competing_species_function(x_val[-1]+step*x3, y_val[-1]+step*y3, parameters)
        
        x_val.append(x_val[-1]+step*(x1+2*x2+2*x3+x4)/6)
        y_val.append(y_val[-1]+step*(y1+2*y2+2*y3+y4)/6)
        t_val.append(t_val[-1]+step)
    return t_val, x_val, y_val